# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 64 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2312.03935


extracting tarball to tmp_2312.03935...

 done.


S. Li  ->  S. Li  |  ['S. Li']


...

Retrieving document from  https://arxiv.org/e-print/2312.04545


extracting tarball to tmp_2312.04545...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2312.04545/bars.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'AppendixC.tex' from 'tmp_2312.04545/AppendixC.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'AppendixB.tex' from 'tmp_2312.04545/AppendixB.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxi

Found 131 bibliographic references in tmp_2312.04545/bars.bbl.
syntax error in line 893: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.04545-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.04545) | **Bar Properties as a Function of Wavelength: A Local Baseline with S4G  for High-Redshift Studies**  |
|| K. Menéndez-Delmestre, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-12-08*|
|*Comments*| *Accepted to MNRAS*|
|**Abstract**| The redshift evolution of bars is an important signpost of the dynamic maturity of disk galaxies. To characterize the intrinsic evolution safe from band-shifting effects, it is necessary to gauge how bar properties vary locally as a function of wavelength. We investigate bar properties in 16 nearby galaxies from the Spitzer Survey of Stellar Structure in Galaxies (S4G) at ultraviolet, optical and mid-infrared wavebands. Based on the ellipticity and position angle profiles from fitting elliptical isophotes to the two-dimensional light distribution, we find that both bar length and ellipticity - the latter often used as a proxy for bar strength - increase at bluer wavebands. Bars are 9% longer in the B-band than at 3.6 um. Their ellipticity increases typically by 8% in the B-band, with a significant fraction (>40%) displaying an increase up to 35%. We attribute the increase in bar length to the presence of star forming knots at the end of bars: these regions are brighter in bluer bands, stretching the bar signature further out. The increase in bar ellipticity could be driven by the apparent bulge size: the bulge is less prominent at bluer bands, allowing for thinner ellipses within the bar region. Alternatively, it could be due to younger stellar populations associated to the bar. The resulting effect is that bars appear longer and thinner at bluer wavebands. This indicates that band-shifting effects are significant and need to be corrected for high-redshift studies to reliably gauge any intrinsic evolution of the bar properties with redshift. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.03935-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.03935) | **The ALMA Survey of 70 μm Dark High-mass Clumps in Early Stages  (ASHES). X: Hot Gas Reveals Deeply Embedded Star Formation**  |
|| N. Izumi, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-12-08*|
|*Comments*| *Accepted for Publication in ApJ. 39 pages, 22 figures, 6 tables*|
|**Abstract**| Massive infrared dark clouds (IRDCs) are considered to host the earliest stages of high-mass star formation. In particular, 70 $\mu$m dark IRDCs are the colder and more quiescent clouds. At a scale of about 5000 au using formaldehyde (H2CO) emission, we investigate the kinetic temperature of dense cores in 12 IRDCs obtained from the pilot ALMA Survey of 70 $\mu$m dark High-mass clumps in Early Stages (ASHES). Compared to 1.3 mm dust continuum and other molecular lines, such as C18O and deuterated species, we find that H2CO is mainly sensitive to low-velocity outflow components rather than to quiescent gas expected in the early phases of star formation. The kinetic temperatures of these components range from 26 to 300 K. The Mach number reaches about 15 with an average value of about 4, suggesting that the velocity distribution of gas traced by H2CO is significantly influenced by a supersonic non-thermal component. In addition, we detect warm line emission from HC3N and OCS in 14 protostellar cores, which requires high excitation temperatures (Eu/k ~ 100 K). These results show that some of the embedded cores in the ASHES fields are in an advanced evolutionary stage, previously unexpected for 70 $\mu$m dark IRDCs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2312.04545.md
    + _build/html/tmp_2312.04545/./Images/Fig2_NGC3049.png
    + _build/html/tmp_2312.04545/./Images/Fig2_NGC1097.png
    + _build/html/tmp_2312.04545/./Images/Fig4_a_emax.png
    + _build/html/tmp_2312.04545/./Images/Fig4_a_edrop.png
    + _build/html/tmp_2312.04545/./Images/Fig4_a_deltapa.png
    + _build/html/tmp_2312.04545/./Images/OptMIR/AppA_NGC0337.png
    + _build/html/tmp_2312.04545/./Images/OptMIR/AppA_NGC1097.png
    + _build/html/tmp_2312.04545/./Images/OptMIR/AppA_NGC1291.png
    + _build/html/tmp_2312.04545/./Images/OptMIR/AppA_NGC1512.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Bar Properties as a Function of Wavelength: A Local Baseline with S$^4$G for High-Redshift Studies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.04545-b31b1b.svg)](https://arxiv.org/abs/2312.04545)<mark>Appeared on: 2023-12-08</mark> -  _Accepted to MNRAS_

</div>
<div id="authors">

K. Menéndez-Delmestre, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** The redshift evolution of bars is an important signpost of the dynamic maturity of disk galaxies. To characterize the intrinsic evolution safe from band-shifting effects, it is necessary to gauge how bar properties vary locally as a function of wavelength. We investigate bar properties in 16 nearby galaxies from the Spitzer Survey of Stellar Structure in Galaxies (S $^4$ G) at ultraviolet, optical and mid-infrared wavebands. Based on the ellipticity and position angle profiles from fitting elliptical isophotes to the two-dimensional light distribution, we find that both bar length and ellipticity -- the latter often used as a proxy for bar strength -- increase at bluer wavebands. Bars are 9 \% longer in the B-band than at 3.6 $\mu$ m. Their ellipticity increases typically by 8 \% in the B-band, with a significant fraction ( $>$ 40 \% ) displaying an increase up to 35 \% . We attribute the increase in bar length to the presence of star forming knots at the end of bars: these regions are brighter in bluer bands, stretching the bar signature further out. The increase in bar ellipticity could be driven by the apparent bulge size: the bulge is less prominent at bluer bands, allowing for thinner ellipses within the bar region. Alternatively, it could be due to younger stellar populations associated to the bar. The resulting effect is that bars appear longer and thinner at bluer wavebands. This indicates that band-shifting effects are significant and need to be corrected for high-redshift studies to reliably gauge any intrinsic evolution of the bar properties with redshift.

</div>

<div id="div_fig1">

<img src="tmp_2312.04545/./Images/Fig2_NGC3049.png" alt="Fig3.1" width="50%"/><img src="tmp_2312.04545/./Images/Fig2_NGC1097.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Ellipticity and position angle radial profiles for NGC 3049 (left) and NGC 1097 (right), the former exemplifying our methodology for the _ higher-resolution optical-through-MIR_ study (including B-, R-, and 3.6$\mu$m bands) and the latter for the _ low-resolution UV-through-MIR_ study (including FUV-, NUV-, B-, R-, and 3.6$\mu$m bands). In both cases we show radial profiles for ellipticity and position angle, as well as their respective variations at each isophotal fit. The vertical down-pointing arrows indicate the semi-major axis (SMA) of maximum ellipticity, i.e., a$_{\epsilon max}$ for each band.  The vertical dashed lines indicate the radial location of the maximum variation in ellipticity (top two panels) and PA (bottom two panels). The dashed portions of the radial profiles indicate regions where the FUV/NUV emission deviates significantly from a smooth light distribution, consequently impacting our isophotal ellipse-fitting approach (see Section \ref{processing}). From top to bottom, postage stamp images in the 3.6$\mu$m, R- and B-bands for NGC 3049 and in the 3.6$\mu$m, R-, B-, NUV- and FUV-bands for NGC 1097 are shown with an overlaid ellipse showing the isophote of maximum ellipticity. Similar figures for all of our sample are shown as part of Appendix \ref{appendixA_profiles_hires} for the _ higher-resolution optical-through-MIR_ study and Appendix \ref{appendixB_profiles_lores} for the _ low-resolution UV-through-MIR_ study.
 (*profiles*)

</div>
<div id="div_fig2">

<img src="tmp_2312.04545/./Images/Fig4_a_emax.png" alt="Fig1.1" width="33%"/><img src="tmp_2312.04545/./Images/Fig4_a_edrop.png" alt="Fig1.2" width="33%"/><img src="tmp_2312.04545/./Images/Fig4_a_deltapa.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Measured bar sizes as a function of wavelength for the galaxies in our sample; each panel corresponds to one of the three bar size definitions adopted for the analysis (see Section \ref{processing} for details). Each galaxy is represented by 3 data points, where the value on the x-axis corresponds to that measured in the bluer band of the waveband pair being considered. We note that the bulk of individual bar size measurements in the B-, R- and 3.6$\mu$m-bands is offset downwards from the identity line (solid line), indicating that bar sizes measured in bluer bands are larger. On average, the bar is measured to be $\sim$9\% longer in the B-band compared to that in the 3.6 $\mu$m. (*length*)

</div>
<div id="div_fig3">

<img src="tmp_2312.04545/./Images/OptMIR/AppA_NGC0337.png" alt="Fig8.1" width="25%"/><img src="tmp_2312.04545/./Images/OptMIR/AppA_NGC1097.png" alt="Fig8.2" width="25%"/><img src="tmp_2312.04545/./Images/OptMIR/AppA_NGC1291.png" alt="Fig8.3" width="25%"/><img src="tmp_2312.04545/./Images/OptMIR/AppA_NGC1512.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** Ellipticity and PA profiles for all galaxies in our sample as part of the _ higher-resolution opt-through-MIR_ study, following the format of Fig. \ref{profiles}. (*appendixA_profiles_hires_fig*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.04545"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

336  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
